In [ ]:
from unsloth import FastLanguageModel
import torch

from transformers import TextStreamer

In [2]:
# base_model = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
base_model = "unsloth/meta-llama-3.1-8b-instruct-bnb-4bit"

max_seq_length = 2048

tuned_model = f"{base_model.replace('/', '_')}-qlora-social-media"

tuned_model_top_dir = f"model_{tuned_model}"
tuned_model_save_dir = f"{tuned_model_top_dir}/model_save"
tuned_model_gguf_dir = f"{tuned_model_top_dir}/model_gguf"
tuned_model_checkpoints_dir = f"{tuned_model_top_dir}/checkpoints"
tuned_model_logs_dir = f"{tuned_model_top_dir}/logs"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=tuned_model_save_dir,
    # model_name=base_model,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
    random_state=42,
)
FastLanguageModel.for_inference(model)

In [ ]:
FastLanguageModel.for_inference(model)

alpaca_prompt = """You are a social media user. Respond to the comment you are shown.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
BOS_TOKEN = tokenizer.bos_token

inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Tube amps are objectively the best amps.",  # prompt
            "",  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=max_seq_length)